In [1]:
!pip install langchain langchain-community langchain-groq langgraph sqlalchemy

## import the library's

In [2]:
import os
import sqlite3
from typing_extensions import TypedDict, Annotated

In [3]:
from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, StateGraph

## # ✅ Set Groq API key

In [ ]:
os.environ["GROQ_API_KEY"] = "give_your_groq_api_key"

NameError: name 'os' is not defined

In [ ]:
# Store your API key
groq_api_key = "give_your_groq_api_key"

## ✅ Connect to ARGO database

## create the llm

In [6]:
# ✅ Limit schema introspection to avoid giant prompts
db = SQLDatabase.from_uri("sqlite:///argo.db")

# ✅ Initialize Groq LLM (Mistral)
# Initialization of Groq LLM with a supported model
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="Gemma2-9b-It",   # use lowercase, as Groq model IDs are case-sensitive
    temperature=0
)
# -------------------------------


## check llm

In [7]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B972B7DD60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B97B6609B0>, model_name='Gemma2-9b-It', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

In [9]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002B97B3EF170>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002B97B3EF170>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002B97B3EF170>),
 QuerySQLCheckerTool(description='Use this tool to 

In [10]:
system_message = """
You are an agent designed to interact with a SQLite database.

The database has 3 main tables:
- floats(id, platform_number)
- profiles(id, float_id, cycle_number, latitude, longitude, time)
- measurements(id, profile_id, pressure, temperature, salinity, time)

Relationships:
- A float can have many profiles (linked by profiles.float_id).
- A profile can have many measurements (linked by measurements.profile_id).

When the user asks about temperature, salinity, or pressure, you must query the
measurements table and JOIN with profiles if filtering by time, latitude, or longitude.

To filter by year or month, use SQLite functions like:
  strftime('%Y', measurements.time) and strftime('%m', measurements.time).

Do not invent tables or columns that do not exist.
""".format(
    dialect="SQLite",
    top_k=5,
)

In [11]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, prompt=system_message)

In [12]:
while True:
    # Take user input
    question = input("\nAsk a question about the database (or type 'exit' to quit): ")

    if question.lower() in ["exit", "quit", "q"]:
        print("Goodbye! 👋")
        break

    # Stream agent responses step by step
    for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()


================================ Human Message =================================


================================== Ai Message ==================================

</tool-use>
================================ Human Message =================================




APIConnectionError: Connection error.